# GOCGEEN Targets with ipyaladin

- author Sylvie Dagoret-Campagne
- creation date : 2025-01-04
- last update : 2025-01-04
- use environnement : conda_py311 (python 3.11)
- inspired ffrom https://github.com/cds-astro/tutorials/blob/master/Past_Conferences_and_Schools/2024-11-14_ADASS/ADASS2024_ipyaladin_demo.ipynb

### Useful links

- [ipyaladin's documentation](https://cds-astro.github.io/ipyaladin/)
- [ipyaladin widgets function](https://cds-astro.github.io/ipyaladin/_collections/notebooks/03_Functions.html)
- [CDS's reporitory of tutorials](https://github.com/cds-astro/tutorials) (they showcase ipyaladin too)

- It may be necessary to access ipyaladin following the instructions here to have the widgets installed : https://github.com/cds-astro/ipyaladin?tab=readme-ov-file#installation

> git clone https://github.com/cds-astro/ipyaladin.git
> 
> cd ipyaladin
> 
> npm install
> 
> npm run dev


and 

> pip install sidecar

## Imports

In [1]:
from ipyaladin import Aladin
from sidecar import Sidecar

import astropy.units as u
from astropy.coordinates import Angle

from astropy.coordinates import SkyCoord

In [2]:
# std lib
from io import BytesIO
import gzip

# 3rd party
import numpy as np
np.seterr(divide='ignore',invalid='ignore')

import pandas as pd
pd.set_option('display.max_columns', 500) # convenient for 

import matplotlib.pyplot as plt
import matplotlib.ticker as plticker

from astropy.io import fits, ascii
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3) # define cosmology

# Data Lab
from dl import queryClient as qc, storeClient as sc

## 1. Configuration

### 1.a Configuration of GOGGREEN

In [3]:
cluster_table = qc.query('select * from gogreen_dr1.clusters',fmt='pandas')
print("Column names:", cluster_table.columns)
cluster_table.head(10)  # first 10 rows of the table

Column names: Index(['cluster', 'fullname', 'cluster_id', 'ra_best', 'dec_best', 'ra_gmos',
       'dec_gmos', 'pa_deg', 'redshift', 'vdisp', 'vdisp_err', 'gogreen_m1',
       'gogreen_m2', 'gogreen_m3', 'gogreen_m4', 'gogreen_m5', 'gogreen_m6',
       'gclass_m1', 'gclass_m2', 'gclass_m3', 'gclass_m4', 'gclass_m5',
       'kphot_cat', 'photoz_cat', 'stelmass_cat', 'image_u', 'image_b',
       'image_g', 'image_v', 'image_r', 'image_i', 'image_z', 'image_j',
       'image_j1', 'image_y', 'image_k', 'image_irac1', 'preimage',
       'random_id'],
      dtype='object')


,cluster,fullname,cluster_id,ra_best,dec_best,ra_gmos,dec_gmos,pa_deg,redshift,vdisp,vdisp_err,gogreen_m1,gogreen_m2,gogreen_m3,gogreen_m4,gogreen_m5,gogreen_m6,gclass_m1,gclass_m2,gclass_m3,gclass_m4,gclass_m5,kphot_cat,photoz_cat,stelmass_cat,image_u,image_b,image_g,image_v,image_r,image_i,image_z,image_j,image_j1,image_y,image_k,image_irac1,preimage,random_id
0,COSMOS-125,COSMOS-125,14,150.6208,2.1675,150.62720,2.159200,270.0,1.40400,NaN,400,GS2015ALP001-02,GS2016ALP001-02,GS2019ALP001-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UVISTA_final_v4.1_t1.fits,UVISTA_final_v4.1_zout.fits,UVISTA_final_colors_sfrs_v4.1.dat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GMICOSMOS-125_pseudo,82.235855
1,COSMOS-221,COSMOS-221,14,150.5620,2.5031,150.57020,2.498600,90.0,1.19600,200.0,50,GS2014BLP001-05,GS2015ALP001-01,GS2016ALP001-01,GS2019ALP001-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UVISTA_final_v4.1_t1.fits,UVISTA_final_v4.1_zout.fits,UVISTA_final_colors_sfrs_v4.1.dat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GMICOSMOS-221_pseudo,16.224028
2,COSMOS-28,COSMOS-28,14,149.4692,1.6685,149.45930,1.675030,78.0,1.31600,285.0,75,GN2015BLP004-03,GN2019ALP004-01,GN2019ALP004-02,GN2019ALP004-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UVISTA_final_v4.1_t1.fits,UVISTA_final_v4.1_zout.fits,UVISTA_final_colors_sfrs_v4.1.dat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GMICOSMOS-28_pseud,14.708872
3,COSMOS-63,COSMOS-63,14,150.3590,1.9352,150.36147,1.928342,165.0,1.17220,NaN,70,GN2015BLP004-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UVISTA_final_v4.1_t1.fits,UVISTA_final_v4.1_zout.fits,UVISTA_final_colors_sfrs_v4.1.dat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GMICOSMOS-63_pseudo,10.737590
4,SPT0205,SPT-CL J0205-5829,1,31.4510,-58.4803,31.43896,-58.482900,90.0,1.32270,678.0,57,GS2014BLP001-06,GS2016BLP001-09,GS2016BLP001-02,GS2017BLP001-03,GS2017BLP001-04,GS2017BDD010-01,NaN,NaN,NaN,NaN,NaN,SPTCL-0205_totalall_FOURSTARKs.cat,SPTCL-0205_totalall_Ks.zout,SPTCL-0205_totalall_Ks.fout,mos_VIMOSU_3,mos_VIMOSB_3,NaN,mos_VIMOSV_3,mos_VIMOSR_3,mos_VIMOSI_3,mos_VIMOSz_3,mos_FOURSTARJ_3,mos_FOURSTARJ1_3,NaN,mos_FOURSTARKs_3,mos_IRAC1_3,mfrgS20140928S0161_add,73.878470
5,SPT0546,SPT-CL J0546-5345,2,86.6403,-53.7610,86.65620,-53.758000,0.0,1.06690,977.0,68,GS2014BLP001-09,GS2014BLP001-10,GS2015BLP001-16,GS2017BLP001-12,GS2015BLP001-15,GS2017BLP001-13,NaN,NaN,NaN,NaN,NaN,SPTCL-0546_totalall_FOURSTARKs.cat,SPTCL-0546_totalall_Ks.zout,SPTCL-0546_totalall_Ks.fout,mos_VIMOSU_3,mos_VIMOSB_3,NaN,mos_VIMOSV_3,mos_VIMOSR_3,mos_VIMOSI_3,mos_VIMOSz_3,mos_FOURSTARJ_3,mos_FOURSTARJ1_3,NaN,mos_FOURSTARKs_3,mos_IRAC1_3,mfrgS20141001S0139_add,60.786003
6,SPT2106,SPT-CL J2106-5844,3,316.5191,-58.7411,316.51913,-58.741100,100.0,1.13066,1055.0,83,GS2018ALP001-01,GS2018ALP001-02,GS2018BLP001-04,GS2019ALP001-01,GS2018BLP001-05,NaN,NaN,NaN,NaN,NaN,NaN,SPTCL-2106_totalall_HAWKIKs.cat,SPTCL-2106_totalall_Ks.zout,SPTCL-2106_totalall_Ks.fout,mos_VIMOSU_3,mos_VIMOSB_3,NaN,mos_VIMOSV_3,mos_VIMOSR_3,mos_VIMOSI_3,mos_VIMOSz_3,mos_FOURSTARJ_3,mos_FOURSTARJ1_3,NaN,mos_FOURSTARKs_3,mos_IRAC1_3,mfrgS20150411S0405_add,56.566788
7,SXDF49,SXDF49XGG,13,34.4996,-5.0649,34.53706,-5.069708,73.0,1.09100,255.0,50,GN2015BLP004-01,GS2018BLP001-01,GS2018BLP001-02,GS2018BLP001-03,GN2018BLP004-01,GS2014BLP001-07,NaN,NaN,NaN,NaN,NaN,SPLASH_SXDF_Mehta_v1p6.fits,SPLASH_SXDF_Mehta_v1p6.fits,SPLASH_SXDF_Mehta_v1p6.fits,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GMISXDF49_pseudo,69.287910
8,SXDF64,SXDF64XGG,13,34.3319,-5.2067,34.32375,-5.171400,180.0,0.91600,530.0,80,GS2014BLP001-08,GS2015BLP001-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SPLASH_SXDF_Mehta_v1p6.fits,SPLASH_SXDF_Mehta_v1p6.fits,SPLASH_SXDF_Mehta_v1p6.fits,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GMISXDF64_pseudo,54.498220
9,SXDF76,SXDF76XGG,13,34.7461,-5.3041,34.73612,-5.321800,350.0,1.45900,520.0,180,GS2014BLP001-02,GS2015BLP001-03,GN2018BLP004-02,GN2018BLP004-03,GN2018BLP004-04,NaN,NaN,NaN,NaN,NaN,NaN,SPLASH_SXDF_Mehta_v1p6.fits,SPLASH_SXDF_Mehta_v1p6.fit

### 1.b Configuration of Aladin

In [4]:
aladin = Aladin()
with Sidecar(title="aladin_output"):
    display(aladin)

In [5]:
# select the FOV of Auxtel (6.7 arcmin)
aladin.fov = Angle(10 * u.arcmin)

Now that the widget is opened on the side, we can resize the window as needed.
The view can be zoomed in and out, or panned. This is a fully functionnal [Aladin Lite](https://github.com/cds-astro/aladin-lite) instance!

## 2. Communication between python and ipyaladin

The added benefit here is that it is connected to the python kernel. We can send information from `ipyaladin` to Python and *vice-versa*.

In [6]:
coo_perseus = SkyCoord.from_name("Perseus Cluster")
aladin.target = coo_perseus

### GOCGREEN cluster Targets

In [7]:
cluster_table.cluster 

0     COSMOS-125
1     COSMOS-221
2      COSMOS-28
3      COSMOS-63
4        SPT0205
5        SPT0546
6        SPT2106
7         SXDF49
8         SXDF64
9         SXDF76
10       SXDF76b
11        SXDF87
12    SpARCS0035
13    SpARCS0219
14    SpARCS0335
15    SpARCS1033
16    SpARCS1034
17    SpARCS1051
18    SpARCS1616
19    SpARCS1634
20    SpARCS1638
21    SpARCS0034
22    SpARCS0036
23    SpARCS0215
24    SpARCS1047
25    SpARCS1613
Name: cluster, dtype: object

In [8]:
cluster_table.fullname 

0            COSMOS-125
1            COSMOS-221
2             COSMOS-28
3             COSMOS-63
4     SPT-CL J0205-5829
5     SPT-CL J0546-5345
6     SPT-CL J2106-5844
7             SXDF49XGG
8             SXDF64XGG
9             SXDF76XGG
10           SXDF76bXGG
11            SXDF87XGG
12    SpARCS J0035-4312
13    SpARCS J0219-0531
14    SpARCS J0335-2929
15    SpARCS J1033+5753
16    SpARCS J1034+5818
17    SpARCS J1051+5818
18    SpARCS J1616+5545
19    SpARCS J1634+4021
20    SpARCS J1638+4038
21    SpARCS J0034-4307
22    SpARCS J0036-4410
23    SpARCS J0215-0343
24    SpARCS J1047+5741
25    SpARCS J1613+5649
Name: fullname, dtype: object

## Show in Aladin

In [18]:
index = 0

In [19]:
ra = cluster_table.iloc[index]["ra_best"]
dec = cluster_table.iloc[index]["dec_best"]

In [20]:
coord = SkyCoord(ra=ra,dec=dec,unit="deg")

In [21]:
aladin.target=coord
aladin.fov = Angle(10.0*u.arcsec)

In [13]:
estimated_dimension = aladin.fov
estimated_dimension

<Angle 0.16666667 deg>

In [14]:
aladin.fov

<Angle 0.16666667 deg>

In [15]:
1*u.arcmin

<Quantity 1. arcmin>